__File: spectroscopic_factors.ipynb__

__Author:   A. J. Tropiano (tropiano.4@osu.edu)__<br/>
__Date:     Oct 8, 2021__

Brief description here.

__Revision history:<br/>__
* xxx xx, xxxx - ...

In [1]:
# Imports

# Set-up

In [ ]:
# Graphics function

In [ ]:
# Run graphics function

In [2]:
# Directories for figures/data/etc.
# Specify nucleus

# Load coordinate-space single-particle wave functions

# Plot coordinate-space single-particle wave functions

In [3]:
# Print normalizations too
# Questions:
# 1. What should I compare too?

# Transform to momentum-space

In [4]:
# Hankel transformation function

In [5]:
# Apply to wave functions

# Plot momentum-space wave functions

In [6]:
# Print normalizations too
# Questions:
# 1. What should I compare too?
# 2. Think of other sanity checks.
# 3. Shouldn't p be able to go negative? I'm using p the same as in k, k' for U(k,k').

# Spectroscopic factors

In [7]:
# Add formulas here: initial and evolved